# Import MANO layer

In [ ]:
import torch
from manopth.manolayer import ManoLayer
from manopth import demo

from commons import *

# Use CUDA device if available

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Initialize the MANO layer

- Hand side: right

In [ ]:
# Create a MANO layer with default settings
mano_layer = ManoLayer(
    center_idx=0,
    flat_hand_mean=True,
    ncomps=45,
    side="right",
    mano_root=PROJ_ROOT/ "config/mano_models",
    use_pca=True,
    root_rot_mode="axisang",
    joint_rot_mode="axisang",
    robust_rot=True,
)

# Move the MANO layer to the device
mano_layer = mano_layer.to(device)

# Load MANO Shape parameters from file

In [ ]:
data = read_data_from_json(PROJ_ROOT / "data/calibration/mano/subject_7/mano.json")
print(f"Load data from json: {data}")

# create a hand pose parameter tensor
shape_params = torch.tensor(data["betas"], dtype=torch.float32).to(device)
print(f"shape_params tensor: {shape_params}")

# add batch dimension
shape_params = shape_params.unsqueeze(0)
print(f"shape_params tensor with batch dim: {shape_params.shape}")

# Load MANO Pose parameters from file

In [ ]:
poses_m = np.load(PROJ_ROOT / "data/recordings/20231022_193630/poses_m.npy")
print(f"MANO poses loaded: {poses_m.shape}")    # (hand_num, frame_num, 51), the first 48 are pose parameters, last 3 are global translation

# read frame_id=230 for right hand (index=0)
pose_m = poses_m[0, 230]
print(f"pose_params: {pose_m}")

# create a hand pose parameter tensor from numpy array
pose_params = torch.tensor(pose_m[:48], dtype=torch.float32).to(device)
print(f"pose_params tensor: {pose_params}")

# create a global translation parameter tensor from numpy array
global_trans = torch.tensor(pose_m[48:], dtype=torch.float32).to(device)
print(f"global_trans tensor: {global_trans}")

# add batch dimension
pose_params = pose_params.unsqueeze(0)
print(f"pose_params tensor with batch dim: {pose_params.shape}")
global_trans = global_trans.unsqueeze(0)
print(f"global_trans tensor with batch dim: {global_trans.shape}")

In [ ]:
hand_verts, hand_joints = mano_layer(pose_params, shape_params, global_trans)
hand_faces = mano_layer.th_faces
# Convert to meters
hand_verts = hand_verts / 1000.0
hand_joints = hand_joints / 1000.0

# Convert the resulting vertices, joints and faces to numpy
hand_verts_np = hand_verts[0].cpu().detach().numpy()
hand_joints_np = hand_joints[0].cpu().detach().numpy()
hand_faces_np = hand_faces.cpu().detach().numpy()

In [ ]:
demo.display_hand(
    {
        'verts': hand_verts,
        'joints': hand_joints
    },
    mano_faces=hand_faces,
)

In [ ]:
# Visualize the hand
# Create a triangle mesh from the vertices and faces
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(hand_verts_np)
mesh.triangles = o3d.utility.Vector3iVector(hand_faces_np)

# Estimate normals
mesh.compute_vertex_normals()
mesh.normalize_normals()

# Paint the mesh a uniform grey color
mesh.paint_uniform_color([0.3, 0.3, 0.3])

# Joints as red spheres
joint_mesh = o3d.geometry.TriangleMesh()
for joint in hand_joints_np:
    joint_mesh += o3d.geometry.TriangleMesh.create_sphere(radius=0.003).translate(joint)
joint_mesh.paint_uniform_color([1, 0, 0])

# Visualize the mesh
o3d.visualization.draw([mesh, joint_mesh])